# Fantasy Baseball Pandas Analysis

This takes a number of inputs and produces fantasy scores for each player.

In [55]:
import requests
import csv
import datetime
import subprocess
import os
import pandas as pd
import numpy as np
import seaborn as sb
from bs4 import BeautifulSoup
import sqlalchemy
import psycopg2
pd.options.display.max_columns = 150

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [25]:
# connection information for the database

POSTGRES_USER = os.environ.get('POSTGRES_USER')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
POSTGRES_IP = "192.168.0.14"
POSTGRES_PORT = 5432
POSTGRES_DB = 'postgres'

## Get Roster Data

This will rip the roster information from ESPN and save it to a local CSV file.

In [3]:
LEAGUE_URL = "http://games.espn.com/flb/leaguerosters?leagueId={league_id}"
LEAGUE_ID = 15594

# translate ESPN names to Fangraphs names.
# Add new names as required, since name is used to join the data sets.
TRANSLATIONS = {
    'Nicky Delmonico': 'Nick Delmonico',
    'Yuli Gurriel': 'Yulieski Gurriel'
}


rosters_html = requests.get(LEAGUE_URL.format(league_id=LEAGUE_ID)).text
rosters_soup = BeautifulSoup(rosters_html, "lxml")

rosters = rosters_soup.find_all("table", {'class': 'playerTableTable'})

players = []
for roster in rosters:
    team_name = roster.find("a").text
    players_html = roster.find_all("td", {'class': 'playertablePlayerName'})
    for player in players_html:
        # parse player info
        player_name = player.text.split(",")[0]
        player_name = player_name.replace("*", "")

        # translate player name if necessary
        translation = TRANSLATIONS.get(player_name)
        if translation:
            player_name = translation

        # add to output list
        players.append([player_name, team_name])

with open("rosters.csv", "w", newline='') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(("Name", "Squad"))
    writer.writerows(players)

## Get and Parse Actuals

Looks through the URLs to grab batting & pitching actuals and deliver those back to the user.

In [4]:
# static urls
season = datetime.datetime.now().year
PITCHERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=c,36,37,38,40,-1,120,121,217,-1,24,41,42,43,44,-1,117,118,119,-1,6,45,124,-1,62,122,13&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_100000".format(season=season)
BATTERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_10000".format(season=season)

# # request the data
pitchers_html = requests.get(PITCHERS_URL).text
batters_html = requests.get(BATTERS_URL).text

Now take the requests and parse out the relevant header information for each of the positions. This function will take one of the fangraphs pages as input and write out a CSV of that information once it's parsed.

In [5]:
def parse_array_from_fangraphs_html(input_html, out_file_name):
    """
    Take a HTML stats page from fangraphs and parse it out to a CSV file.
    """
    # parse input
    soup = BeautifulSoup(input_html, "lxml")
    table = soup.find("table", {"class": "rgMasterTable"})
    
    # get headers
    headers_html = table.find("thead").find_all("th")
    headers = []
    for header in headers_html:
        headers.append(header.text)
    
    # get rows
    rows = []
    rows_html = table.find("tbody").find_all("tr")
    for row in rows_html:
        row_data = []
        for cell in row.find_all("td"):
            row_data.append(cell.text)
        rows.append(row_data)
    
    # write to CSV file
    with open(out_file_name, "w") as out_file:
        writer = csv.writer(out_file)
        writer.writerow(headers)
        writer.writerows(rows)

Now that we have all of the player data, I'm writing these out to a CSV file if I want to load them again later without having to run the requests to those pages once more.

In [6]:
parse_array_from_fangraphs_html(batters_html, 'batters_actuals.csv')
parse_array_from_fangraphs_html(pitchers_html, 'pitchers_actuals.csv')

## Get Projections

For this part, we need to call some external bash code here, because the form data is too big to reasonably bring into the script here. Check out the [original blog post](https://zmsy.co/blog/fantasy-baseball/) on how to configure this for your own purposes.

In [10]:
subprocess.call('./get_fangraphs.sh', shell=True)

0

## Read Data Into Pandas

Load those CSV files using read_csv() in pandas. Since some of the percentage values are stored as strings, we need to parse those into floats.

We want to create two dataframes here:

- `dfb` - Batters Dataframe
- `dfp` - Pitchers Dataframe

In [11]:
df_rost = pd.read_csv('rosters.csv')
dfb_act = pd.read_csv('batters_actuals.csv')
dfp_act = pd.read_csv('pitchers_actuals.csv')

# create a function to parse out percentage strings to floats
def parse_pctg(value):
    return float(value.split()[0]) / 100

# apply that to all percentage values in the dataframes
dfb_act['BB%'] = dfb_act['BB%'].apply(lambda x: parse_pctg(x))
dfb_act['K%'] = dfb_act['K%'].apply(lambda x: parse_pctg(x))
dfp_act['K%'] = dfp_act['K%'].apply(lambda x: parse_pctg(x))
dfp_act['BB%'] = dfp_act['BB%'].apply(lambda x: parse_pctg(x))
dfp_act['K-BB%'] = dfp_act['K-BB%'].apply(lambda x: parse_pctg(x))
dfp_act['LOB%'] = dfp_act['LOB%'].apply(lambda x: parse_pctg(x))

In [12]:
with open('batters_projections.html', 'r') as bhtml:
    btxt = bhtml.read()
    dfb_proj = pd.read_html(btxt)[-1]  # read_html returns ALL tables, we just want the last one.
    dfb_proj.dropna(axis=1, inplace=True)

with open('pitchers_projections.html', 'r') as phtml:
    ptxt = phtml.read()
    dfp_proj = pd.read_html(ptxt)[-1]
    dfp_proj.dropna(axis=1, inplace=True)

In [13]:
# join the datasets together. we want one
# for batters and one for pitchers, with
# roster information in both of them.

dfb = pd.merge(dfb_proj, df_rost, how='left', on='Name', suffixes=('.p', '.r'))
dfb = pd.merge(dfb, dfb_act, how='left', on='Name', suffixes=('', '.a'))

dfp = pd.merge(dfp_proj, df_rost, how='left', on='Name', suffixes=('.p', '.r'))
dfp = pd.merge(dfp, dfp_act, how='left', on='Name', suffixes=('', '.a'))

## Filter and Qualify Information

The dataframes for pitchers/batters contain a lot of noise for things that we don't really care about, or won't actually have much of an effect on our league.

In [27]:
# apply some filters so we can get rid of players who won't play.
# minimum plate appearances or innings pitched

dfb = dfb[dfb['PA'] > 100]
dfp = dfp[dfp['IP'] > 20]

# add in league information
LEAGUES = {
    'Angels': 'AL',
    'Astros': 'AL',
    'Athletics': 'AL',
    'Blue Jays': 'AL',
    'Braves': 'NL',
    'Brewers': 'NL',
    'Cardinals': 'NL',
    'Cubs': 'NL',
    'Diamondbacks': 'NL',
    'Dodgers': 'NL',
    'Giants': 'NL',
    'Indians': 'AL',
    'Mariners': 'AL',
    'Marlins': 'NL',
    'Mets': 'NL',
    'Nationals': 'NL',
    'Orioles': 'AL',
    'Padres': 'NL',
    'Phillies': 'NL',
    'Pirates': 'NL',
    'Rangers': 'AL',
    'Rays': 'AL',
    'Red Sox': 'AL',
    'Reds': 'NL',
    'Rockies': 'NL',
    'Royals': 'AL',
    'Tigers': 'AL',
    'Twins': 'AL',
    'White Sox': 'AL',
    'Yankees': 'AL'
}


# derive the league for each player. my league is AL-only, so we want to focus on that.
def league(x):
    return LEAGUES.get(x)

dfb['League'] = dfb['Team'].apply(lambda x: league(x))
dfp['League'] = dfp['Team'].apply(lambda x: league(x))

In [44]:
# rearrange columns for readability and filter some out
# keep only ones relevant for our league
dfb_columns = ['#', 'Name', 'Squad', 'Team', 'League', 'PA', 'PA.a', 'AB', 'H', 'SO', 'K%', 'HR', 'HR.a', 'AVG',
     'ISO', 'BABIP', 'wRC+', 'AVG.a', 'OBP', 'OBP.a', 'wOBA', 'wOBA.a', 'SLG', 'SLG.a', 'OPS', 'BB%', 'BB']

dfp_columns = ['#', 'Name', 'Squad', 'Team', 'League', 'IP', 'ERA', 'ER', 'HR', 'SO', 'BB', 'WHIP', 'K/9', 'BB/9', 'FIP',
     'K/9.a', 'BB/9.a', 'K/BB', 'K%', 'WHIP.a', 'SO.a', 'LOB%', 'ERA.a', 'FIP.a', 'E-F', 'xFIP', 'SIERA', 'IP.a']


dfb = dfb[dfb_columns]
dfp = dfp[dfp_columns]

## Calculate Scores

The individual players in both the batting and pitching groups will get scored based on the entirety of the sample available. We calculate a composite score by taking the individual z-scores in each of the categories and trying to determine which players are above average.

In [45]:
# a 1 represents a positive number, i.e. higher is better
# a -1 represents negative, meaning lower is better

dfb_score_cols = {
    'PA': 1,
    'K%': -1,
    'HR': 1,
    'ISO': 1,
    'OBP': 1,
    'wOBA': 1,
    'SLG': 1
}

dfp_score_cols = {
    'IP': 1,
    'ERA': -1,
    'HR': -1,
    'SO': 1,
    'WHIP': -1,
    'FIP': -1,
    'K/9': 1
}

for col in dfb_score_cols.keys():
    col_score = col + "_score"
    dfb[col_score] = (dfb[col] - dfb[col].mean()) / dfb[col].std(ddof=0) * dfb_score_cols.get(col)
    

for col in dfp_score_cols.keys():
    col_score = col + "_score"
    dfp[col_score] = (dfp[col] - dfp[col].mean()) / dfp[col].std(ddof=0) * dfp_score_cols.get(col)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [46]:
dfp

,#,Name,Squad,Team,League,IP,ERA,ER,HR,SO,BB,WHIP,K/9,BB/9,FIP,K/9.a,BB/9.a,K/BB,K%,WHIP.a,SO.a,LOB%,ERA.a,FIP.a,E-F,xFIP,SIERA,IP.a,IP_score,ERA_score,SO_score,WHIP_score,FIP_score,K/9_score,HR_score
2,81.0,Chris Sale,BEANTOWN ALL THE WAY DOWN,Red Sox,AL,49.0,2.62,14,5,65,10,0.97,11.94,1.90,2.59,13.50,2.03,6.64,0.387,0.85,219.0,0.835,1.97,1.96,0.02,2.27,2.26,146.0,1.567174,2.889319,3.060398,3.216136,2.800901,2.763600,0.008348
11,114.0,Clayton Kershaw,NaN,Dodgers,NL,48.0,2.81,15,5,52,8,1.02,9.79,1.55,2.86,8.89,1.56,5.68,0.248,1.02,108.0,0.810,2.47,2.95,-0.48,3.01,3.28,109.1,1.445499,2.573081,1.829639,2.790013,2.364228,1.227291,0.008348
12,91.0,Max Scherzer,NaN,Nationals,NL,51.0,2.95,17,6,67,13,1.03,12.00,2.32,2.87,12.06,2.11,5.71,0.345,0.89,234.0,0.848,2.11,2.59,-0.48,3.02,2.71,174.2,1.810524,2.340064,3.249746,2.704788,2.348055,2.806474,-0.722113
13,73.0,Jacob deGrom,NaN,Mets,NL,52.0,2.99,17,5,61,13,1.09,10.50,2.31,2.89,11.04,2.04,5.42,0.313,0.97,195.0,0.825,1.81,2.14,-0.32,2.73,2.90,159.0,1.932200,2.273488,2.681703,2.193440,2.315709,1.734630,0.008348
15,224.0,Noah Syndergaard,NaN,Mets,NL,44.0,3.16,15,4,48,10,1.12,9.89,2.06,2.90,9.39,1.88,5.00,0.245,1.25,105.0,0.738,3.40,2.68,0.71,3.14,3.34,100.2,0.958799,1.990538,1.450944,1.937766,2.299536,1.298747,0.738809
22,137.0,Corey Kluber,DIRTY WATER,Indians,AL,55.0,3.25,20,6,59,11,1.09,9.71,1.76,3.13,8.57,1.18,7.27,0.248,0.92,160.0,0.792,2.68,3.31,-0.63,3.19,3.32,168.0,2.297225,1.840741,2.492355,2.193440,1.927555,1.170126,-0.722113
24,272.0,James Paxton,BEANTOWN ALL THE WAY DOWN,Mariners,AL,33.0,3.27,12,4,39,9,1.15,10.43,2.50,3.14,11.37,2.39,4.76,0.314,1.11,176.0,0.755,3.68,3.13,0.55,3.09,3.04,139.1,-0.379626,1.807453,0.598879,1.682092,1.911382,1.684611,0.738809
29,309.0,Stephen Strasburg,NaN,Nationals,NL,38.0,3.37,14,4,44,10,1.13,10.59,2.29,3.17,10.65,2.21,4.81,0.288,1.15,101.0,0.716,3.90,3.50,0.40,3.04,3.17,85.1,0.228749,1.641012,1.072248,1.852541,1.862862,1.798941,0.738809
35,98.0,Aaron Nola,NaN,Phillies,NL,44.0,3.40,17,4,46,12,1.19,9.45,2.43,3.25,8.94,2.46,3.64,0.254,0.98,160.0,0.810,2.24,2.73,-0.50,3.34,3.54,161.0,0.958799,1.591080,1.261596,1.341193,1.733478,0.984340,0.738809
39,210.0,Carlos Carrasco,THE RAINMAKERS,Indians,AL,48.0,3.29,18,6,53,11,1.11,9.86,2.01,3.29,10.11,1.76,5.75,0.279,1.10,161.0,0.754,3.33,3.08,0.25,3.12,3.14,143.1,1.445499,1.774165,1.924313,2.022990,1.668785,1.277311,-0.722113


## Write Information Back to Database

Once the table is available in the database, then we can query it again using other tools to make our lives easier. Then it can be used to display the information about each player in the Superset DB.

In [54]:
conn = sqlalchemy.create_engine("psycopg2+postgres://{user}:{password}@{host}:{port}/{db}".format(
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_IP,
    port=POSTGRES_PORT,
    db=POSTGRES_DB
))

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:psycopg2.postgres